# LLM Market Analytics Examples

This notebook demonstrates how to use the LLM-based market forecaster for analyzing and predicting market movements.

## Load and Prepare Sample Data

Let's create some sample market data for demonstration purposes.

In [1]:
from quantlib.datasource.service import DataService
data_service = DataService.create_default_service()

# Define parameters for data fetching
symbol = "IBOV"  # Change to your desired symbol
start_date = "2021-01-01"  # Adjust as needed
end_date = "2025-05-16"    # Adjust as needed
interval = "1m"            # 1-minute data

ModuleNotFoundError: No module named 'MetaTrader5'

In [10]:
ts_object = data_service.get_close_prices(
        symbol=symbol,
        source='mt5',
        start_date=start_date,
        end_date=end_date,
        interval=interval
    )

In [15]:
ts_object.data.to_parquet('petr_returns.parquet')

## Use the data and build on unix enviroment

In [2]:
import pandas as pd
import numpy as np
import torch
from quantlib.analytics.llm import LLMForecaster, MarketTokenizer
from quantlib.data.timeseries import TimeSeries

In [3]:
ts = TimeSeries(pd.read_parquet('quantlib/notebooks/win_returns.parquet'))

In [4]:
ret = ts.returns(intraday_only=True)
returns = ret.data[(ret.data.index.hour >= 10) & (ret.data.index.hour < 17)]

In [5]:
# 2. Initialize the forecaster
forecaster = LLMForecaster()


In [6]:
# 3. Prepare the dataset
forecaster.build_dataset(returns['close'])

In [ ]:
# 4. Build and train the model
forecaster.build_model()
forecaster.train()

Epoch 1, Loss: 0.2077


In [32]:
# 5. Generate predictions
# Create a prompt from recent market data
recent_data = returns['close'].iloc[-60:].values
prompt = torch.tensor(
    MarketTokenizer.series_to_tokens(
        pd.Series(recent_data),
        forecaster.vocab_size
    )[0],
    dtype=torch.long
).unsqueeze(0)  # Add batch dimension

# Generate predictions
predicted_tokens = forecaster.fit(prompt, steps=5)  # Predict next 5 days

# Convert predictions back to returns
predicted_returns = MarketTokenizer.tokens_to_values(
    np.array(predicted_tokens),
    forecaster.bins
)

print("Predicted returns for next periods:", predicted_returns[:-5])

IndexError: index 127 is out of bounds for axis 0 with size 127

In [27]:
# 6. Evaluate predictions (example with simple metrics)
actual_returns = returns.iloc[-5:].values
mse = np.mean((predicted_returns - actual_returns) ** 2)
print(f"Mean Squared Error: {mse:.6f}")

Mean Squared Error: 0.000031
